In [37]:
import os

%pwd 

'c:\\Users\\dell\\Documents\\GitHub\\Kidney-Tumor-Classification'

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [39]:
from kidneyClassifier import *
from kidneyClassifier.constants import *
from kidneyClassifier.utils.common import read_yaml, create_directories
from config import *

In [40]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
                
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                
                create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
               root_dir=config.root_dir,
               source_URL=config.source_URL,
               local_data_file = config.local_data_file,
               unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


In [41]:
import os
import urllib.request as request
import zipfile
import gdown
from kidneyClassifier import logger
from kidneyClassifier.utils.common import get_size


In [42]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''Fetch Data from the url'''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok = True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/us?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir ,quiet=False, fuzzy=True)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e


    def extract_zip_file(self):
        '''
        zip_file_path: str
        extracts the zip file into the data directory
        Function returns None
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)

        if not zipfile.is_zipfile(self.config.local_data_file): 
            raise ValueError(f"The file {self.config.local_data_file} is not a valid ZIP file.")


        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-08-27 20:47:21,634: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-27 20:47:21,645: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-27 20:47:21,655: INFO: common: created directory at: artifacts]
[2024-08-27 20:47:21,664: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-27 20:47:21,671: INFO: 3073340465: Downloading data from https://drive.google.com/file/d/1-2Y6sOXk3eLNwpFxXDZCmbSneG39xJie/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/us?/export=download&id=1-2Y6sOXk3eLNwpFxXDZCmbSneG39xJie
To: c:\Users\dell\Documents\GitHub\Kidney-Tumor-Classification\artifacts\data_ingestion\data.zip
168B [00:00, 26.1kB/s]


[2024-08-27 20:47:43,634: INFO: 3073340465: Downloaded data from https://drive.google.com/file/d/1-2Y6sOXk3eLNwpFxXDZCmbSneG39xJie/view?usp=sharing into file artifacts/data_ingestion/data.zip]


ValueError: The file artifacts/data_ingestion/data.zip is not a valid ZIP file.